In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os
from dotenv import load_dotenv,find_dotenv
import pandas as pd

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence

In [11]:
_=load_dotenv(find_dotenv())

In [12]:
llm_model="gemini-1.5-flash"

In [13]:
df=pd.read_csv('data.csv')

In [14]:
llm=ChatGoogleGenerativeAI(tempurature=0.9,model=llm_model)

In [15]:
first_prompt=ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}"
)

In [16]:
second_prompt=ChatPromptTemplate.from_template(
    "Write a 20-word description for the following compant: {company_name}"
)

In [17]:
chain_one=first_prompt | llm
chain_two=second_prompt | llm

In [18]:
overall_simple_chain=RunnableSequence(chain_one | chain_two)

In [19]:
product="Queen Size Sheet Set"

In [20]:
response=overall_simple_chain.invoke({"product":product})

In [21]:
print(response)

content='Luxury bedding brand offering superior comfort and quality.  Names evoke serenity, relaxation, and regal elegance.  Perfect for discerning customers.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-a4783d06-0678-4dbe-b85d-33b3ddfadc7b-0' usage_metadata={'input_tokens': 471, 'output_tokens': 27, 'total_tokens': 498, 'input_token_details': {'cache_read': 0}}


In [26]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import SequentialChain,LLMChain

In [27]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [28]:
first_prompt=ChatPromptTemplate.from_template(
    "Translate the following review to English:\n\n{Review}"
)

In [29]:
chain_one=LLMChain(llm=llm,prompt=first_prompt,output_key="English_Review")

In [30]:
second_prompt=ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:\n\n{English_Review}"
)

In [31]:
chain_two=LLMChain(llm=llm,prompt=second_prompt,output_key="summary")

In [32]:
third_prompt=ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

In [34]:
chain_three=LLMChain(llm=llm,prompt=third_prompt,output_key="language")

In [35]:
fourth_prompt=ChatPromptTemplate.from_template(
    "Write a follow-up response to the folllowing summary in the specified language:\n\Summary: {summary}\n\nLanguage: {language}"
)

In [36]:
chain_four=LLMChain(llm=llm,prompt=fourth_prompt,output_key="followup_message")

In [41]:
overall_chain=SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables=["Review"],
    output_variables=["English_Review","summary","followup_message"],
    verbose=True
)

In [42]:
df=pd.DataFrame({"Review":["Esta es una reseña de ejemplo.", "Ceci est un avis d'exemple."]})

In [43]:
review=df.Review[0]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Esta es una reseña de ejemplo.',
 'English_Review': 'This is an example review.',
 'summary': 'This is a placeholder review with no content to summarize.',
 'followup_message': 'Pues entonces, ¡no hay nada que comentar!  No hay resumen que seguir.  ¿Deberíamos buscar una reseña real para poder discutirla?'}

In [45]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [51]:
 prompt_infos=[
    {
        "name":"physics",
        "description":"Good for answering questions about physics.",
        "prompt_template":physics_template
    },
    {
        "name":"math",
        "description":"Good for answering math questions.",
        "prompt_template":math_template
    },
    {
        "name":"history",
        "description":"Good for answering questions about history.",
        "prompt_template":history_template
    },
    {
        "name":"computerscience",
        "description":"Good for answering questions about computer science.",
        "prompt_template":computerscience_template
    }
]

In [52]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [53]:
llm=ChatGoogleGenerativeAI(temperature=0.1,model=llm_model)

In [55]:
destination_chains={}
for p_info in prompt_infos:
    name=p_info["name"]
    prompt_template=p_info["prompt_template"]
    prompt=ChatPromptTemplate.from_template(template=prompt_template)
    chain=LLMChain(llm=llm,prompt=prompt)
    destination_chains[name]=chain

destinations=[f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str="\n".join(destinations)

In [56]:
default_prompt=ChatPromptTemplate.from_template("{input}")
default_chain=LLMChain(llm=llm,prompt=default_prompt)

In [57]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [58]:
router_template=MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt=PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain=LLMRouterChain.from_llm(llm,router_prompt)

In [59]:
chain=MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [60]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by an idealized object called a black body.  A black body absorbs all electromagnetic radiation incident upon it, regardless of frequency or angle of incidence.  Because it absorbs all radiation, it also emits radiation at all frequencies, with the intensity of that emission at each frequency determined solely by its temperature.  The spectrum of this emitted radiation is described by Planck's law."

In [61]:
chain.run("What is AI")



> Entering new MultiPromptChain chain...
None: {'input': 'What is AI?'}
> Finished chain.


'Artificial intelligence (AI) is a broad field encompassing the development of computer systems capable of performing tasks that typically require human intelligence.  These tasks include things like:\n\n* **Learning:** Acquiring information and rules for using the information.\n* **Reasoning:** Using rules to reach approximate or definite conclusions.\n* **Problem-solving:** Finding solutions to complex situations.\n* **Perception:** Interpreting sensory information like images, sound, and text.\n* **Language understanding:** Processing and generating human language.\n\nAI systems achieve these capabilities through various techniques, including:\n\n* **Machine learning (ML):** Algorithms that allow systems to learn from data without explicit programming.  This includes subfields like deep learning (using artificial neural networks with many layers) and reinforcement learning (learning through trial and error).\n* **Natural language processing (NLP):** Enabling computers to understand,

In [62]:
chain.run("What is the capital of France?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the capital of France?'}
> Finished chain.


'Paris'

In [63]:
chain.run("What is 1+1")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 1+1?'}
> Finished chain.


'The question is: What is 1 + 1?\n\nThis problem can be broken down into the following component parts:\n\n* **Understanding the operands:**  We have two operands: 1 and 1.  Both are integers representing the quantity "one".\n\n* **Understanding the operator:** The operator "+" signifies addition, which is the mathematical operation of combining quantities.\n\n* **Performing the addition:**  Combining the quantity "one" with the quantity "one" results in the quantity "two".\n\nTherefore, the answer to the broader question, 1 + 1, is $\\boxed{2}$.'

In [64]:
chain.run("What is operating system?")



> Entering new MultiPromptChain chain...
computerscience: {'input': 'What is an operating system?'}
> Finished chain.


"An operating system (OS) is a complex piece of software that acts as an intermediary between a computer's hardware and the applications you use.  It manages and controls all the hardware and software resources of a computer system.  Think of it as the conductor of an orchestra, ensuring all the different instruments (hardware components) play together harmoniously to produce the desired output (running applications).\n\nHere's a breakdown of its key functions, described in imperative steps a machine could interpret (though a real OS is far more intricate):\n\n1. **Boot Process:**  Upon power-on, the OS initiates a boot sequence. This involves:\n    1.1. **Power-On Self Test (POST):**  Checking hardware components for functionality.\n    1.2. **BIOS/UEFI Initialization:** Loading the basic input/output system (BIOS) or Unified Extensible Firmware Interface (UEFI), which initializes hardware and loads the boot loader.\n    1.3. **Boot Loader Execution:** Loading the OS kernel into memor